# Mushroom classifier



# Libraries

In [0]:
!pip install -q pydot
!pip install graphviz 
!apt-get install graphviz

import os
import json
import keras

import pickle
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt


from PIL import Image
from tqdm import tqdm
from random import shuffle
from keras import optimizers
from keras import callbacks
from keras.utils import plot_model


from IPython.display import clear_output
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator
from google.colab import drive

drive.mount('/gdrive')

HEIGHT = 224
WIDTH = 224
BATCH_SIZE = 256
EPOCHS = 100

# Model
Import ResNet (image classification) with weights from ImageNet but without top layer.

In [0]:
def create_model(freeze_layers_from='base_model'):
  base_model = ResNet50(weights='imagenet', include_top=False,input_shape=(224,224,3))
  # Add final layers
  x = base_model.output
  x = Flatten()(x)
  predictions = Dense(16, activation='softmax', name='fc1000')(x)

  model = Model(input=base_model.input, output=predictions)
  print('Number of layers: {}'.format(len(model.layers)))
  # Freeze some layers
  if freeze_layers_from is not None:
      if freeze_layers_from == 'base_model':
          print ('   Freezing base model layers')
          for layer in base_model.layers:
              layer.trainable = False
      else:
          for i, layer in enumerate(model.layers):
              print(i, layer.name)
          print ('   Freezing from layer 0 to ' + str(freeze_layers_from))
          for layer in model.layers[:freeze_layers_from]:
             layer.trainable = False
          for layer in model.layers[freeze_layers_from:]:
             layer.trainable = True
  
  opt = optimizers.Adam(lr=0.0001)
  
  model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
  return model

# Load dataset

Training split:
- 2076 images  
- 39 classes


In [0]:
def resize(input_image=224):
  input_image = Image.open(input_image).resize((HEIGHT, WIDTH),Image.LANCZOS)
  return input_image

In [0]:
!unzip  /gdrive/'My Drive'/dataset_small
!ls /gdrive/'My Drive'
TRAIN_SIZE= 7415
VALIDATION_SIZE= 1301

In [0]:
# data generator
datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        zoom_range=0.2,
        rotation_range=20,
        width_shift_range=0.2,
		    height_shift_range=0.2,
        shear_range=0.2,
        validation_split=0.15,
    	  horizontal_flip=True,
    vertical_flip=True,
		    fill_mode="nearest"
        )


# this is a generator that will read pictures found in
# subfolders of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = datagen.flow_from_directory(
        'dataset', # this is the target directory
        shuffle=True,
        seed=82,
        interpolation='bicubic',
        subset='training',
        target_size=(224,224),
        batch_size=BATCH_SIZE,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = datagen.flow_from_directory(
        'dataset',
        batch_size=BATCH_SIZE,
        target_size=(224,224),
        shuffle=False,
        subset='validation',
        interpolation='bicubic',
        class_mode='categorical')
            

# Callbacks

In [0]:
reduce_lr= callbacks.ReduceLROnPlateau(monitor='val_loss', patience=10)
early_stopping= callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
checkpoint= callbacks.ModelCheckpoint('best_model.hdf5',monitor='val_loss',save_best_only=True)

# Training

In [0]:
model = create_model()

model.fit_generator(
        train_generator,
        steps_per_epoch=TRAIN_SIZE // BATCH_SIZE,
        epochs=EPOCHS,
        callbacks=[reduce_lr, early_stopping, checkpoint],
        verbose=1,
        validation_data=validation_generator,
        validation_steps=VALIDATION_SIZE //BATCH_SIZE)

model.save('classifier.h5')

# clear_output()


In [0]:
label_map = (train_generator.class_indices)
print(label_map)

# Evaluate dataset

In [0]:
def test_model(model, img):
  image = resize(img)
  x = np.expand_dims(image, axis=0) 
  x = preprocess_input(x)
  print('Result: {}'.format(np.argmax(model.predict(x))))

In [0]:
test_model(model, './dataset/amanita_muscaria/amanita_muscaria_150.jpg')